# Recognize planar markers using template matching

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np


In [39]:
marker1 = cv2.imread('images/ex3_1.jpg')
marker2 = cv2.imread('images/ex3_1b.jpg')

print(marker1.shape)


cornersMarker1 = np.array([[ 41, 29],
 [597, 52],
 [617, 564],
 [ 39, 557]], dtype=np.float32)


(589, 665, 3)


In [40]:
img1 = cv2.imread('images/ex3_2.jpg')


def order_points(pts):

	rect = np.zeros((4, 2), dtype = "float64")

	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]

	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

# Define the callback function for mouse events
def mouse_callback(event, x, y, flags, params):
    # If left button is clicked, record the point coordinates
    if event == cv2.EVENT_LBUTTONDOWN:
        try:
            params['points'].append([x, y])
            cv2.circle(params['img_select_points'], (x, y), 5, (0, 0, 255), -1)
        except:
            print('Invalid input')
            params['points'].pop()
            cv2.circle(params['img_select_points'], (x, y), 5, (0, 0, 0), -1)


# Create a window and set the mouse callback function
def select_points_from_image(img_src, no_of_points=4):
    img_select_points = img_src.copy()
    cv2.namedWindow('image')
    params = {'points': [], 'real_coords': [], 'img_select_points': img_select_points}
    cv2.setMouseCallback('image', mouse_callback, params)

    # Display the image and wait for user to select points
    while True:
        cv2.imshow('image', img_select_points)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if len(params['points']) == no_of_points:
            break

    selected_pts = np.array(params['points']) 

    # Print the selected points
    print('Selected points:', selected_pts)
    cv2.destroyAllWindows()
    return selected_pts


cornersImg1 = order_points(select_points_from_image(img1, 4))


Selected points: [[385 222]
 [602 275]
 [339 343]
 [567 389]]


In [4]:
H, _ = cv2.findHomography(cornersImg1, cornersMarker1)
print(H)


[[ 2.03809170e+00  7.79711886e-01 -9.27881311e+02]
 [-7.78002735e-01  3.58686794e+00 -4.63705288e+02]
 [-2.84359577e-04  8.69592868e-05  1.00000000e+00]]


In [45]:
offset = 1500
img1_warp = cv2.warpPerspective(img1, H, (img1.shape[1] + offset , img1.shape[0] + offset ))

img1_window = cv2.resize(img1_warp, (960, 540)) 
cv2.imshow('Warp prespective image',img1_window)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [47]:
result_marker1 = img1_warp.copy()
marker1_gray = cv2.cvtColor(marker1, cv2.COLOR_BGR2GRAY)
img1_warp_gray = cv2.cvtColor(img1_warp, cv2.COLOR_BGR2GRAY)
w, h = marker1_gray.shape[::-1]

# Apply template Matching
res = cv2.matchTemplate(img1_warp_gray,marker1_gray,cv2.TM_CCOEFF_NORMED)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
top_left = max_loc

bottom_right = (top_left[0] + w , top_left[1] + h)
cv2.rectangle(result_marker1, top_left, bottom_right, (0, 0, 255), 5)

result_marker1_window = cv2.resize(result_marker1, (960, 540)) 
cv2.imshow('Detected point',result_marker1_window)

cv2.waitKey(0)

cv2.destroyAllWindows()

In [48]:
result_marker2 = result_marker1.copy()
marker2_gray = cv2.cvtColor(marker2, cv2.COLOR_BGR2GRAY)
img1_warp_gray = cv2.cvtColor(result_marker1, cv2.COLOR_BGR2GRAY)
w, h = marker1_gray.shape[::-1]

# Apply template Matching
res = cv2.matchTemplate( img1_warp_gray, marker2_gray,cv2.TM_CCOEFF_NORMED)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
top_left = max_loc

bottom_right = (top_left[0] + w , top_left[1] + h)
cv2.rectangle(result_marker2, top_left, bottom_right, (0, 255, 0), 5)

result_window = cv2.resize(result_marker2, (960, 540)) 
cv2.imshow('Detected point',result_window)

cv2.waitKey(0)

cv2.destroyAllWindows()

In [49]:
IH, _ = cv2.findHomography(cornersMarker1, cornersImg1)
img1_back = cv2.warpPerspective(result_marker2, IH, (result_marker2.shape[1] - offset, result_marker2.shape[0] - offset))

# Check number of channels
img1_back_window = cv2.resize(img1_back, (960, 540)) 
cv2.imshow('Detected point', img1_back_window)

cv2.waitKey(0)

cv2.destroyAllWindows()